In [1]:
import pandas as pd
import numpy as np
import requests
import json

import matplotlib.pyplot as plt
from scipy.stats import linregress

from pathlib import Path

## Importing Data

In [2]:
metacritic_path = Path("Resources/metacritic.csv")
metacritic_data = pd.read_csv(metacritic_path)
metacritic_data.head()

,id,name,year,metacritic_rating,reviewer_rating,positivity_ratio,to_beat_main,to_beat_extra,to_beat_completionist,extra_content_length,tags
0,96000,The Tiny Bang Story,2011.0,63.0,8.0,7.073879,3.60,3.60,3.77,0.17,Story Rich|Steampunk|Adventure|Atmospheric|Puz...
1,262410,World of Guns: Gun Disassembly,2014.0,NaN,8.0,5.208940,2.00,NaN,28.07,26.07,Horror|First-Person|Historical|Multiplayer|Str...
2,1250410,Microsoft Flight Simulator 40th Anniversary Ed...,2020.0,91.0,6.0,3.581082,NaN,NaN,NaN,NaN,Multiplayer|Adventure|VR|Action Roguelike|Phys...
3,365450,Hacknet,2015.0,82.0,8.0,14.548520,7.06,8.73,10.75,3.69,Horror|Story Rich|Hacking|Crime|Multiplayer|Dy...
4,92800,SpaceChem,2011.0,84.0,8.0,11.440415,43.32,57.79,67.55,24.23,Automation|Strategy|Building|Puzzle|Science|Pr...


In [3]:
metacritic_count = len(metacritic_data)
metacritic_count

63543

In [4]:
genre_path = Path("Resources/genre.csv")
genre_data = pd.read_csv(genre_path)
genre_data.head()

,Unnamed: 0,game,link,release,peak_players,positive_reviews,negative_reviews,total_reviews,rating,primary_genre,store_genres,publisher,developer,detected_technologies,store_asset_mod_time,review_percentage,players_right_now,24_hour_peak,all_time_peak,all_time_peak_date
0,0,Pizza Tower,/app/2231450/,2023-01-26,4529,19807,227,20034,96.39,Action (1),"Action (1), Indie (23)",Tour De Pizza,Tour De Pizza,Engine.GameMaker; SDK.FMOD,2023-01-10,98.0,"3,101","3,606",4529,2023-03-25
1,1,Resident Evil 4,/app/2050650/,2023-03-24,168191,61752,1616,63368,95.75,Action (1),"Action (1), Adventure (25)",CAPCOM Co. Ltd.,CAPCOM Co. Ltd.,Engine.RE_Engine; SDK.AMD_GPU_Services,2023-03-29,97.0,"46,289","47,164",168191,2023-03-25
2,2,The Murder of Sonic the Hedgehog,/app/2324650/,2023-03-31,15543,12643,213,12856,95.54,Casual (4),"Casual (4), Free to Play (37), Indie (23), Sim...",SEGA,SEGA,Engine.Unity; SDK.UnityBurst; SDK.cURL,2023-03-24,98.0,618,776,15543,2023-03-31
3,3,Pineapple on pizza,/app/2263010/,2023-03-28,1415,11717,209,11926,95.39,Indie (23),"Casual (4), Free to Play (37), Indie (23)",Majorariatto,Majorariatto,Engine.Unity; SDK.cURL,2023-03-22,98.0,101,101,1415,2023-03-28
4,4,Hi-Fi RUSH,/app/1817230/,2023-01-25,6132,14152,324,14476,95.09,Action (1),Action (1),Bethesda Softworks,Tango Gameworks,Engine.Unreal; SDK.Intel_OID; SDK.Intel_XeSS; ...,2023-04-11,97.0,630,664,6132,2023-01-29


In [5]:
genre_count = len(genre_data)
genre_count

67571

In [6]:
metacritic_reduced = metacritic_data[['name','year','metacritic_rating','reviewer_rating','positivity_ratio','tags']]
metacritic_reduced

,name,year,metacritic_rating,reviewer_rating,positivity_ratio,tags
0,The Tiny Bang Story,2011.0,63.0,8.0,7.073879,Story Rich|Steampunk|Adventure|Atmospheric|Puz...
1,World of Guns: Gun Disassembly,2014.0,NaN,8.0,5.208940,Horror|First-Person|Historical|Multiplayer|Str...
2,Microsoft Flight Simulator 40th Anniversary Ed...,2020.0,91.0,6.0,3.581082,Multiplayer|Adventure|VR|Action Roguelike|Phys...
3,Hacknet,2015.0,82.0,8.0,14.548520,Horror|Story Rich|Hacking|Crime|Multiplayer|Dy...
4,SpaceChem,2011.0,84.0,8.0,11.440415,Automation|Strategy|Building|Puzzle|Science|Pr...
...,...,...,...,...,...,...
63538,Uncharted Waters,2017.0,NaN,7.0,6.000000,Retro|RPG|Trading
63539,Kid Chameleon™,2010.0,NaN,NaN,6.000000,Retro|Platformer
63540,Fatal Labyrinth™,2010.0,NaN,7.0,5.000000,RPG
63541,Sangokushi Eiketsuden,2017.0,NaN,6.0,3.153846,RPG


In [7]:
metacritic_reduced = metacritic_reduced.rename(columns = {'name': 'game'})
metacritic_reduced

,game,year,metacritic_rating,reviewer_rating,positivity_ratio,tags
0,The Tiny Bang Story,2011.0,63.0,8.0,7.073879,Story Rich|Steampunk|Adventure|Atmospheric|Puz...
1,World of Guns: Gun Disassembly,2014.0,NaN,8.0,5.208940,Horror|First-Person|Historical|Multiplayer|Str...
2,Microsoft Flight Simulator 40th Anniversary Ed...,2020.0,91.0,6.0,3.581082,Multiplayer|Adventure|VR|Action Roguelike|Phys...
3,Hacknet,2015.0,82.0,8.0,14.548520,Horror|Story Rich|Hacking|Crime|Multiplayer|Dy...
4,SpaceChem,2011.0,84.0,8.0,11.440415,Automation|Strategy|Building|Puzzle|Science|Pr...
...,...,...,...,...,...,...
63538,Uncharted Waters,2017.0,NaN,7.0,6.000000,Retro|RPG|Trading
63539,Kid Chameleon™,2010.0,NaN,NaN,6.000000,Retro|Platformer
63540,Fatal Labyrinth™,2010.0,NaN,7.0,5.000000,RPG
63541,Sangokushi Eiketsuden,2017.0,NaN,6.0,3.153846,RPG


In [8]:
metacritic_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63543 entries, 0 to 63542
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   game               63543 non-null  object 
 1   year               63443 non-null  float64
 2   metacritic_rating  3916 non-null   float64
 3   reviewer_rating    44629 non-null  float64
 4   positivity_ratio   61530 non-null  float64
 5   tags               63543 non-null  object 
dtypes: float64(4), object(2)
memory usage: 2.9+ MB


In [9]:
metacritic_reduced['year'] = metacritic_reduced['year'].astype('Int64')

# Check the updated data types
metacritic_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63543 entries, 0 to 63542
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   game               63543 non-null  object 
 1   year               63443 non-null  Int64  
 2   metacritic_rating  3916 non-null   float64
 3   reviewer_rating    44629 non-null  float64
 4   positivity_ratio   61530 non-null  float64
 5   tags               63543 non-null  object 
dtypes: Int64(1), float64(3), object(2)
memory usage: 3.0+ MB


In [10]:
genre_reduced = genre_data[['game','release','rating','primary_genre','store_genres','publisher','developer',]]
genre_reduced

,game,release,rating,primary_genre,store_genres,publisher,developer
0,Pizza Tower,2023-01-26,96.39,Action (1),"Action (1), Indie (23)",Tour De Pizza,Tour De Pizza
1,Resident Evil 4,2023-03-24,95.75,Action (1),"Action (1), Adventure (25)",CAPCOM Co. Ltd.,CAPCOM Co. Ltd.
2,The Murder of Sonic the Hedgehog,2023-03-31,95.54,Casual (4),"Casual (4), Free to Play (37), Indie (23), Sim...",SEGA,SEGA
3,Pineapple on pizza,2023-03-28,95.39,Indie (23),"Casual (4), Free to Play (37), Indie (23)",Majorariatto,Majorariatto
4,Hi-Fi RUSH,2023-01-25,95.09,Action (1),Action (1),Bethesda Softworks,Tango Gameworks
...,...,...,...,...,...,...,...
67566,Space Empires V,2006-10-16,67.34,Strategy (2),Strategy (2),Strategy First,Malfador Machinations
67567,Iron Warriors: T-72 Tank Command,2006-07-26,60.17,Strategy (2),Strategy (2),Strategy First,Strategy First
67568,Gumboy: Crazy Adventures,2006-12-19,50.32,Unknown Genre (0),"Casual (4), Indie (23)",CINEMAX s.r.o.,CINEMAX s.r.o.
67569,Shattered Union,2006-10-25,50.27,Strategy (2),Strategy (2),2K,PopTop


In [11]:
genre_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67571 entries, 0 to 67570
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   game           67571 non-null  object 
 1   release        67571 non-null  object 
 2   rating         67571 non-null  float64
 3   primary_genre  67561 non-null  object 
 4   store_genres   67514 non-null  object 
 5   publisher      67110 non-null  object 
 6   developer      67443 non-null  object 
dtypes: float64(1), object(6)
memory usage: 3.6+ MB


In [12]:
# Assuming 'metacritic_reduced' is your DataFrame
genre_reduced['release'] = pd.to_datetime(genre_reduced['release']).dt.year.astype('Int64')

# Check the updated data types
genre_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67571 entries, 0 to 67570
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   game           67571 non-null  object 
 1   release        67571 non-null  Int64  
 2   rating         67571 non-null  float64
 3   primary_genre  67561 non-null  object 
 4   store_genres   67514 non-null  object 
 5   publisher      67110 non-null  object 
 6   developer      67443 non-null  object 
dtypes: Int64(1), float64(1), object(5)
memory usage: 3.7+ MB


/var/folders/tf/l4q9ccl500b9zqywywq4xw7h0000gn/T/ipykernel_67595/1508223190.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre_reduced['release'] = pd.to_datetime(genre_reduced['release']).dt.year.astype('Int64')


In [13]:
genre_reduced

,game,release,rating,primary_genre,store_genres,publisher,developer
0,Pizza Tower,2023,96.39,Action (1),"Action (1), Indie (23)",Tour De Pizza,Tour De Pizza
1,Resident Evil 4,2023,95.75,Action (1),"Action (1), Adventure (25)",CAPCOM Co. Ltd.,CAPCOM Co. Ltd.
2,The Murder of Sonic the Hedgehog,2023,95.54,Casual (4),"Casual (4), Free to Play (37), Indie (23), Sim...",SEGA,SEGA
3,Pineapple on pizza,2023,95.39,Indie (23),"Casual (4), Free to Play (37), Indie (23)",Majorariatto,Majorariatto
4,Hi-Fi RUSH,2023,95.09,Action (1),Action (1),Bethesda Softworks,Tango Gameworks
...,...,...,...,...,...,...,...
67566,Space Empires V,2006,67.34,Strategy (2),Strategy (2),Strategy First,Malfador Machinations
67567,Iron Warriors: T-72 Tank Command,2006,60.17,Strategy (2),Strategy (2),Strategy First,Strategy First
67568,Gumboy: Crazy Adventures,2006,50.32,Unknown Genre (0),"Casual (4), Indie (23)",CINEMAX s.r.o.,CINEMAX s.r.o.
67569,Shattered Union,2006,50.27,Strategy (2),Strategy (2),2K,PopTop


In [14]:
genre_reduced = genre_reduced.rename(columns = {'release': 'year'})
genre_reduced

,game,year,rating,primary_genre,store_genres,publisher,developer
0,Pizza Tower,2023,96.39,Action (1),"Action (1), Indie (23)",Tour De Pizza,Tour De Pizza
1,Resident Evil 4,2023,95.75,Action (1),"Action (1), Adventure (25)",CAPCOM Co. Ltd.,CAPCOM Co. Ltd.
2,The Murder of Sonic the Hedgehog,2023,95.54,Casual (4),"Casual (4), Free to Play (37), Indie (23), Sim...",SEGA,SEGA
3,Pineapple on pizza,2023,95.39,Indie (23),"Casual (4), Free to Play (37), Indie (23)",Majorariatto,Majorariatto
4,Hi-Fi RUSH,2023,95.09,Action (1),Action (1),Bethesda Softworks,Tango Gameworks
...,...,...,...,...,...,...,...
67566,Space Empires V,2006,67.34,Strategy (2),Strategy (2),Strategy First,Malfador Machinations
67567,Iron Warriors: T-72 Tank Command,2006,60.17,Strategy (2),Strategy (2),Strategy First,Strategy First
67568,Gumboy: Crazy Adventures,2006,50.32,Unknown Genre (0),"Casual (4), Indie (23)",CINEMAX s.r.o.,CINEMAX s.r.o.
67569,Shattered Union,2006,50.27,Strategy (2),Strategy (2),2K,PopTop


In [15]:
merged_df = pd.merge(metacritic_reduced, genre_reduced, how = 'outer', 
                     left_on = ['game', 'year'], right_on = ['game', 'year'])
merged_df

,game,year,metacritic_rating,reviewer_rating,positivity_ratio,tags,rating,primary_genre,store_genres,publisher,developer
0,The Tiny Bang Story,2011,63.0,8.0,7.073879,Story Rich|Steampunk|Adventure|Atmospheric|Puz...,84.85,Adventure (25),"Adventure (25), Casual (4), Indie (23)",Colibri Games,Colibri Games
1,World of Guns: Gun Disassembly,2014,NaN,8.0,5.208940,Horror|First-Person|Historical|Multiplayer|Str...,82.09,Free to Play (37),"Action (1), Casual (4), Free to Play (37), Ind...",Noble Empire Corp.,Noble Empire Corp.
2,Microsoft Flight Simulator 40th Anniversary Ed...,2020,91.0,6.0,3.581082,Multiplayer|Adventure|VR|Action Roguelike|Phys...,NaN,NaN,NaN,NaN,NaN
3,Hacknet,2015,82.0,8.0,14.548520,Horror|Story Rich|Hacking|Crime|Multiplayer|Dy...,91.41,Simulation (28),"Indie (23), Simulation (28)",Fellow Traveller,Team Fractal Alligator
4,SpaceChem,2011,84.0,8.0,11.440415,Automation|Strategy|Building|Puzzle|Science|Pr...,88.72,Simulation (28),"Indie (23), Simulation (28)",Zachtronics,Zachtronics
...,...,...,...,...,...,...,...,...,...,...,...
78178,BloodRayne 2,2006,NaN,NaN,NaN,NaN,68.50,Action (1),Action (1),Ziggurat,Terminal Reality
78179,The Ship Single Player,2006,NaN,NaN,NaN,NaN,67.99,RPG (3),"Action (1), Indie (23), RPG (3)",Blazing Griffin Ltd.,Outerlight Ltd.
78180,Race: The WTCC Game,2006,NaN,NaN,NaN,NaN,67.85,Racing (9),Racing (9),SimBin,SimBin
78181,Iron Warriors: T-72 Tank Command,2006,NaN,NaN,NaN,NaN,60.17,Strategy (2),Strategy (2),Strategy First,Strategy First


In [16]:
(merged_df['tags']).head(10)

0    Story Rich|Steampunk|Adventure|Atmospheric|Puz...
1    Horror|First-Person|Historical|Multiplayer|Str...
2    Multiplayer|Adventure|VR|Action Roguelike|Phys...
3    Horror|Story Rich|Hacking|Crime|Multiplayer|Dy...
4    Automation|Strategy|Building|Puzzle|Science|Pr...
5    Horror|First-Person|Crime|Multiplayer|Adventur...
6    Multiplayer|Strategy|Action|Adventure|Physics|...
7    Horror|Sokoban|Strategy|Level Editor|Pixel Gra...
8    Robots|Co-op Campaign|Automobile Sim|Action-Ad...
9    Procedural Generation|Strategy|Soundtrack|Atmo...
Name: tags, dtype: object

In [17]:
merged_df.dtypes

game                  object
year                   Int64
metacritic_rating    float64
reviewer_rating      float64
positivity_ratio     float64
tags                  object
rating               float64
primary_genre         object
store_genres          object
publisher             object
developer             object
dtype: object

In [18]:
#Split the "list" into lists
merged_df['tags_list'] = merged_df['tags'].str.split('|')
merged_df['tags_list'] = merged_df['tags_list'].astype('object')  # Update the data type to 'object'

print(merged_df.dtypes)


game                  object
year                   Int64
metacritic_rating    float64
reviewer_rating      float64
positivity_ratio     float64
tags                  object
rating               float64
primary_genre         object
store_genres          object
publisher             object
developer             object
tags_list             object
dtype: object


In [19]:
(merged_df['tags_list']).head(10)

0    [Story Rich, Steampunk, Adventure, Atmospheric...
1    [Horror, First-Person, Historical, Multiplayer...
2    [Multiplayer, Adventure, VR, Action Roguelike,...
3    [Horror, Story Rich, Hacking, Crime, Multiplay...
4    [Automation, Strategy, Building, Puzzle, Scien...
5    [Horror, First-Person, Crime, Multiplayer, Adv...
6    [Multiplayer, Strategy, Action, Adventure, Phy...
7    [Horror, Sokoban, Strategy, Level Editor, Pixe...
8    [Robots, Co-op Campaign, Automobile Sim, Actio...
9    [Procedural Generation, Strategy, Soundtrack, ...
Name: tags_list, dtype: object

In [20]:
merged_df.head()

,game,year,metacritic_rating,reviewer_rating,positivity_ratio,tags,rating,primary_genre,store_genres,publisher,developer,tags_list
0,The Tiny Bang Story,2011,63.0,8.0,7.073879,Story Rich|Steampunk|Adventure|Atmospheric|Puz...,84.85,Adventure (25),"Adventure (25), Casual (4), Indie (23)",Colibri Games,Colibri Games,"[Story Rich, Steampunk, Adventure, Atmospheric..."
1,World of Guns: Gun Disassembly,2014,NaN,8.0,5.208940,Horror|First-Person|Historical|Multiplayer|Str...,82.09,Free to Play (37),"Action (1), Casual (4), Free to Play (37), Ind...",Noble Empire Corp.,Noble Empire Corp.,"[Horror, First-Person, Historical, Multiplayer..."
2,Microsoft Flight Simulator 40th Anniversary Ed...,2020,91.0,6.0,3.581082,Multiplayer|Adventure|VR|Action Roguelike|Phys...,NaN,NaN,NaN,NaN,NaN,"[Multiplayer, Adventure, VR, Action Roguelike,..."
3,Hacknet,2015,82.0,8.0,14.548520,Horror|Story Rich|Hacking|Crime|Multiplayer|Dy...,91.41,Simulation (28),"Indie (23), Simulation (28)",Fellow Traveller,Team Fractal Alligator,"[Horror, Story Rich, Hacking, Crime, Multiplay..."
4,SpaceChem,2011,84.0,8.0,11.440415,Automation|Strategy|Building|Puzzle|Science|Pr...,88.72,Simulation (28),"Indie (23), Simulation (28)",Zachtronics,Zachtronics,"[Automation, Strategy, Building, Puzzle, Scien..."


In [21]:
#Drop the 'tags'
merged_df.drop('tags', axis=1, inplace=True)

merged_df.head()

,game,year,metacritic_rating,reviewer_rating,positivity_ratio,rating,primary_genre,store_genres,publisher,developer,tags_list
0,The Tiny Bang Story,2011,63.0,8.0,7.073879,84.85,Adventure (25),"Adventure (25), Casual (4), Indie (23)",Colibri Games,Colibri Games,"[Story Rich, Steampunk, Adventure, Atmospheric..."
1,World of Guns: Gun Disassembly,2014,NaN,8.0,5.208940,82.09,Free to Play (37),"Action (1), Casual (4), Free to Play (37), Ind...",Noble Empire Corp.,Noble Empire Corp.,"[Horror, First-Person, Historical, Multiplayer..."
2,Microsoft Flight Simulator 40th Anniversary Ed...,2020,91.0,6.0,3.581082,NaN,NaN,NaN,NaN,NaN,"[Multiplayer, Adventure, VR, Action Roguelike,..."
3,Hacknet,2015,82.0,8.0,14.548520,91.41,Simulation (28),"Indie (23), Simulation (28)",Fellow Traveller,Team Fractal Alligator,"[Horror, Story Rich, Hacking, Crime, Multiplay..."
4,SpaceChem,2011,84.0,8.0,11.440415,88.72,Simulation (28),"Indie (23), Simulation (28)",Zachtronics,Zachtronics,"[Automation, Strategy, Building, Puzzle, Scien..."


In [22]:
#Drop the (num) and create lists
merged_df['store_genres'] = (
    merged_df['store_genres']
    .apply(lambda x: [genre.split(' ')[0] for genre in str(x).strip('[]').split(', ')] if pd.notna(x) else 'NaN')
)

merged_df.head()

,game,year,metacritic_rating,reviewer_rating,positivity_ratio,rating,primary_genre,store_genres,publisher,developer,tags_list
0,The Tiny Bang Story,2011,63.0,8.0,7.073879,84.85,Adventure (25),"[Adventure, Casual, Indie]",Colibri Games,Colibri Games,"[Story Rich, Steampunk, Adventure, Atmospheric..."
1,World of Guns: Gun Disassembly,2014,NaN,8.0,5.208940,82.09,Free to Play (37),"[Action, Casual, Free, Indie, Simulation, Stra...",Noble Empire Corp.,Noble Empire Corp.,"[Horror, First-Person, Historical, Multiplayer..."
2,Microsoft Flight Simulator 40th Anniversary Ed...,2020,91.0,6.0,3.581082,NaN,NaN,NaN,NaN,NaN,"[Multiplayer, Adventure, VR, Action Roguelike,..."
3,Hacknet,2015,82.0,8.0,14.548520,91.41,Simulation (28),"[Indie, Simulation]",Fellow Traveller,Team Fractal Alligator,"[Horror, Story Rich, Hacking, Crime, Multiplay..."
4,SpaceChem,2011,84.0,8.0,11.440415,88.72,Simulation (28),"[Indie, Simulation]",Zachtronics,Zachtronics,"[Automation, Strategy, Building, Puzzle, Scien..."


In [23]:
#Remove (num) from each genre
merged_df['primary_genre'] = (
    merged_df['primary_genre']
    .apply(lambda x: ', '.join([genre.split(' ')[0] for genre in str(x).strip('[]').split(', ')]) if pd.notna(x) else 'NaN')
)

merged_df

,game,year,metacritic_rating,reviewer_rating,positivity_ratio,rating,primary_genre,store_genres,publisher,developer,tags_list
0,The Tiny Bang Story,2011,63.0,8.0,7.073879,84.85,Adventure,"[Adventure, Casual, Indie]",Colibri Games,Colibri Games,"[Story Rich, Steampunk, Adventure, Atmospheric..."
1,World of Guns: Gun Disassembly,2014,NaN,8.0,5.208940,82.09,Free,"[Action, Casual, Free, Indie, Simulation, Stra...",Noble Empire Corp.,Noble Empire Corp.,"[Horror, First-Person, Historical, Multiplayer..."
2,Microsoft Flight Simulator 40th Anniversary Ed...,2020,91.0,6.0,3.581082,NaN,NaN,NaN,NaN,NaN,"[Multiplayer, Adventure, VR, Action Roguelike,..."
3,Hacknet,2015,82.0,8.0,14.548520,91.41,Simulation,"[Indie, Simulation]",Fellow Traveller,Team Fractal Alligator,"[Horror, Story Rich, Hacking, Crime, Multiplay..."
4,SpaceChem,2011,84.0,8.0,11.440415,88.72,Simulation,"[Indie, Simulation]",Zachtronics,Zachtronics,"[Automation, Strategy, Building, Puzzle, Scien..."
...,...,...,...,...,...,...,...,...,...,...,...
78178,BloodRayne 2,2006,NaN,NaN,NaN,68.50,Action,[Action],Ziggurat,Terminal Reality,NaN
78179,The Ship Single Player,2006,NaN,NaN,NaN,67.99,RPG,"[Action, Indie, RPG]",Blazing Griffin Ltd.,Outerlight Ltd.,NaN
78180,Race: The WTCC Game,2006,NaN,NaN,NaN,67.85,Racing,[Racing],SimBin,SimBin,NaN
78181,Iron Warriors: T-72 Tank Command,2006,NaN,NaN,NaN,60.17,Strategy,[Strategy],Strategy First,Strategy First,NaN


In [24]:
merged_df.head()

,game,year,metacritic_rating,reviewer_rating,positivity_ratio,rating,primary_genre,store_genres,publisher,developer,tags_list
0,The Tiny Bang Story,2011,63.0,8.0,7.073879,84.85,Adventure,"[Adventure, Casual, Indie]",Colibri Games,Colibri Games,"[Story Rich, Steampunk, Adventure, Atmospheric..."
1,World of Guns: Gun Disassembly,2014,NaN,8.0,5.208940,82.09,Free,"[Action, Casual, Free, Indie, Simulation, Stra...",Noble Empire Corp.,Noble Empire Corp.,"[Horror, First-Person, Historical, Multiplayer..."
2,Microsoft Flight Simulator 40th Anniversary Ed...,2020,91.0,6.0,3.581082,NaN,NaN,NaN,NaN,NaN,"[Multiplayer, Adventure, VR, Action Roguelike,..."
3,Hacknet,2015,82.0,8.0,14.548520,91.41,Simulation,"[Indie, Simulation]",Fellow Traveller,Team Fractal Alligator,"[Horror, Story Rich, Hacking, Crime, Multiplay..."
4,SpaceChem,2011,84.0,8.0,11.440415,88.72,Simulation,"[Indie, Simulation]",Zachtronics,Zachtronics,"[Automation, Strategy, Building, Puzzle, Scien..."
